<a href="https://colab.research.google.com/github/katieirenec/code-from-stackoverflow/blob/main/TuringTarpit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://stackapi.readthedocs.io/en/latest/
!pip install stackapi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from stackapi import StackAPI

#connecting to stackoverflow api
SITE = StackAPI('stackoverflow')

#https://api.stackexchange.com/docs/advanced-search
#edit the default filter to get the fielda you want, which generates a code for that filter, which is used in the way below
questionwithanswers = SITE.fetch('search/advanced', tagged = 'Python', sort='votes', filter='!Oeu9xT2P(F9w-bQY2Nfb7(L.P2DjuORRH9YOE3.z7*9')

In [ ]:
type(questionwithanswers)

dict

In [ ]:
#example question with a list of answers
questionwithanswers["items"][0]

{'tags': ['python', 'iterator', 'generator'],
 'answers': [{'tags': ['python', 'iterator', 'generator'],
   'score': 615,
   'answer_id': 6400990,
   'question_id': 231767,
   'body': '\n<p>The <code>yield</code> keyword is reduced to two simple facts:</p>\n<ol>\n<li>If the compiler detects the <code>yield</code> keyword <em>anywhere</em> inside a function, that function no longer returns via the <code>return</code> statement. <em><strong>Instead</strong></em>, it <strong>immediately</strong> returns a <strong>lazy &quot;pending list&quot; object</strong> called a generator</li>\n<li>A generator is iterable. What is an <em>iterable</em>? It\'s anything like a <code>list</code> or <code>set</code> or <code>range</code> or dict-view, with a <em>built-in protocol for visiting each element in a certain order</em>.</li>\n</ol>\n<p>In a nutshell: Most commonly, <strong>a generator is a lazy, incrementally-pending list</strong>, and <strong><code>yield</code> statements allow you to use funct

In [ ]:
stack_list = []

#for each question item
for item in questionwithanswers["items"]:
  #if the score of the question > 10, it is accepted
  if (item["score"] < 10): break;
  stack = {}
  stack["question_body"] = item["body"]
  stack["question_title"] = item["title"]
  stack["question_id"] = item["question_id"]
  answer_list = []
  #iterates through all answers for the question
  for ans in item["answers"]:
    #for each answer in the question, the score should be > 0
    if (ans["score"] < 0): 
      break;
    stack_ans = {}
    stack_ans["answer_body"] = ans["body"]
    stack_ans["answer_id"] = ans["answer_id"]
    answer_list.append(stack_ans)
  stack["answer"] = answer_list
  #stores the body, title, question_id for each question along with the list of answers and their answer id, answer body
  stack_list.append(stack)

In [ ]:
#an example of answer_body for one answer
stack_list[0]["answer"][0]["answer_body"]

'<p>A JavaScript solution for Internet&nbsp;Explorer is:</p>\n\n<pre><code>onselectstart="return false;"\n</code></pre>\n'

In [ ]:
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup

questions_parsed = []

#goes through each question object
for stack in stack_list:
  question = {}
  #uses beautiful soup to parse the body
  parsed_html = BeautifulSoup(stack["question_body"])
  question["question_id"] = stack["question_id"]
  question["question_body"] = parsed_html.body.text
  #finds all the code tags in the body and stores the text between them separately
  question_code_list = parsed_html.body.find_all('code')
  question["question_code"] = []
  for code in question_code_list:
    question["question_code"].append(code.text)
  answer = []
  #iterates through all answers for the question
  for ans in stack["answer"]:
    answer_check = {}
    parsed_answer = BeautifulSoup(ans["answer_body"])
    answer_code_list = parsed_answer.body.find_all('code')
    answer_check["answer_code"] = []
    for code in answer_code_list:
      answer_check["answer_code"].append(code.text)
    answer_check["answer_body"] = parsed_answer.body.text
    answer_check["answer_id"] = ans["answer_id"]
    #stores all the answers in a list
    answer.append(answer_check)
  question["answer"] = answer
  questions_parsed.append(question)

In [ ]:
#an example of the parsed data structure
questions_parsed[0]

{'question_id': 826782,
 'question_body': 'For anchors that act like buttons (for example, the buttons on the sidebar of this Stack\xa0Overflow page titled Questions, Tags, and Users) or tabs, is there a CSS standard way to disable the highlighting effect if the user accidentally selects the text?\nI realize that this could be done with JavaScript and a little googling yielded the Mozilla-only -moz-user-select option.\nIs there a standard-compliant way to accomplish this with CSS, and if not, what is the "best practice" approach?\n',
 'question_code': ['-moz-user-select'],
 'answer': [{'answer_code': ['onselectstart="return false;"\n'],
   'answer_body': 'A JavaScript solution for Internet\xa0Explorer is:\nonselectstart="return false;"\n\n',
   'answer_id': 1730172},
  {'answer_code': ['-moz-user-select', '-webkit-user-select'],
   'answer_body': 'Until CSS 3\'s user-select property becomes available, Gecko-based browsers support the -moz-user-select property you already found. WebKit 

In [ ]:
import csv
import json
import sys

orig_stdout = sys.stdout

filename = "code1.csv"
csvfile = open(filename, "w")
#changes the stdout to the file
sys.stdout = csvfile
for question in questions_parsed:
  #prints each question in the file
  print(question)

#reverts back to stdout
sys.stdout = orig_stdout
#closes the file
csvfile.close()